In [1]:
import keras

Using TensorFlow backend.


In [2]:
import cv2
import os, gc, sys, glob
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn import model_selection
from sklearn import metrics

from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator

In [3]:
train_set = pd.read_csv('input/train_labels.csv')
test_set = pd.read_csv('input/sample_submission.csv')

In [4]:
train_set.head()

,name,invasive
0,1,0
1,2,0
2,3,1
3,4,0
4,5,1


In [5]:
test_set.head()

,name,invasive
0,1,0.5
1,2,0.5
2,3,0.5
3,4,0.5
4,5,0.5


In [6]:
img_size1 = 128
img_size2 = 128
def read_img(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (img_size1, img_size2))
    return img

train_img, test_img = [], []
for img_path in tqdm(train_set['name'].iloc[: ]):
    train_img.append(read_img('input/train/' + str(img_path) + '.jpg'))
for img_path in tqdm(test_set['name'].iloc[: ]):
    test_img.append(read_img('input/test/' + str(img_path) + '.jpg'))

100%|██████████| 1531/1531 [02:33<00:00,  8.65it/s]


In [7]:
train_img = np.array(train_img, np.float32) / 255
train_label = np.array(train_set['invasive'].iloc[: ])
test_img = np.array(test_img, np.float32) / 255

In [12]:
def model_nn():
    model = Sequential()
    model.add(Conv2D(8, (3, 3), activation='relu', input_shape=(img_size1, img_size2, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(16, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(Flatten())
    #change 2048 to 1024 to speed up
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(0.65))
    #change 512 to 256
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.55))
    model.add(Dense(1, activation='sigmoid'))
    #change learning rate from 0.001 to 0.002
    #sgd = optimizers.SGD(lr = 0.001, decay = 1e-6, momentum = 0.8, nesterov = True) 
    adam = optimizers.Adam(lr = 0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics=['accuracy'])
    print(model.summary())
    return model

#change from 8 to 4
n_fold = 8
kf = model_selection.KFold(n_splits = n_fold, shuffle = True)
eval_fun = metrics.roc_auc_score

In [13]:
def run_oof(tr_x, tr_y, te_x, kf):
    preds_train = np.zeros(len(tr_x), dtype = np.float)
    preds_test = np.zeros(len(te_x), dtype = np.float)
    train_loss = []; test_loss = []

    i = 1
    for train_index, test_index in kf.split(tr_x):
    #train_index, test_index = kf.split(tr_x)        
        x_tr = tr_x[train_index]; x_te = tr_x[test_index]
        y_tr = tr_y[train_index]; y_te = tr_y[test_index]

        datagen = ImageDataGenerator(
            # featurewise_center = True,
            rotation_range = 30,
            width_shift_range = 0.2,
            height_shift_range = 0.2,
            # zca_whitening = True,
            shear_range = 0.2,
            zoom_range = 0.2,
            horizontal_flip = True,
            vertical_flip = True,
            fill_mode = 'nearest')
        datagen.fit(x_tr)

        model = model_nn()
        earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience = 15, verbose=0, mode='auto')
        model.fit_generator(datagen.flow(x_tr, y_tr, batch_size = 64),
            validation_data = (x_te, y_te), callbacks = [earlystop],
            steps_per_epoch = len(train_img) / 64, epochs = 500, verbose = 2)

        train_loss.append(eval_fun(y_tr, model.predict(x_tr)[:, 0]))
        test_loss.append(eval_fun(y_te, model.predict(x_te)[:, 0]))

        preds_train[test_index] = model.predict(x_te)[:, 0]
        preds_test += model.predict(te_x)[:, 0]

        print('{0}: Train {1:0.5f} Val {2:0.5f}'.format(i, train_loss[-1], test_loss[-1]))
        i += 1
        #break

    print('Train: ', train_loss)
    print('Val: ', test_loss)
    print('Train{0:0.5f}_Test{1:0.5f}\n\n'.format(np.mean(train_loss), np.mean(test_loss)))
    preds_test /= n_fold
    return preds_train, preds_test

In [14]:
train_pred, test_pred = run_oof(train_img, train_label, test_img, kf)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 126, 126, 8)       224       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 63, 63, 8)         0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 61, 61, 16)        1168      
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 30, 30, 16)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 28, 28, 32)        4640      
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 12, 12, 64)        18496     
__________

In [15]:
test_set['invasive'] = test_pred
test_set.to_csv('./submit-1.csv', index = None)